# Putting it all together

Let's build a simple blockchain using all the tools we learnt about.

In [1]:
import json
import hashlib
from time import time
from pprint import pprint

To get started we define two transactions to get us rolling and see everything working.

In [2]:
# Two transactions: we send 10 coins from A to B and then 3 from B to C.
transaction1 = {'from': "A's address",
                'to': "B's address",
                'amount': 10}
transaction2 = {'from': "B's address",
                'to': "C's address",
                'amount': 3}

## Grouping transactions into blocks

A new block has a timestamp and links to the previous one via its hash. Otherwise the list of transactions, the hash and nonce are empty. We will set them when we mine the block.

In [3]:
def new_block(previous_block):
    if not previous_block['hash']:
        raise RuntimeError("You need to close a block by mining it before it can be extended")

    block = {'timestamp': time(),
             'transactions': [],
             'nonce': 0,
             'hash': '',
             'previous_hash': previous_block['hash']
             }
    return block

There is one special block: the genesis block. So far we avoided talking about how to get your  blockchain started, this is how. The genesis block is similar to the others except it does not refer to a previous one, the `previous_hash` field is empty.

In [4]:
def genesis_block():
    return {'timestamp': time(),
            'transactions': [],
            'nonce': 0,
            'hash': '',
            'previous_hash': ''
            }

## Mining a new block

Performing the proof-of-work and thereby "locking" the transactions in it in place is called mining. Why? Because the node that performs the verification and proof-of-work is rewarded with a coin. The transaction looks a little different from others because it originates from a special source: the network.

This is how coins come to exist in our blockchain.

In [5]:
BLOCKCHAIN = []

def mine(block, miner):
    block['transactions'].append({'from': 'network', 'to': miner, 'amount': 1})
    h = ''
    while not h.startswith('0000'):
        block['nonce'] += 1
        block_string = json.dumps(block, sort_keys=True).encode()
        h = hashlib.sha256(block_string).hexdigest()

    block['hash'] = h
    BLOCKCHAIN.append(block)

    return block

## Building up our chain

Let's put these to work: generate the genesis block and then create one new block that contains our two transactions:

In [6]:
first_block = genesis_block()
print('first block:')
pprint(first_block)

first block:
{'hash': '',
 'nonce': 0,
 'previous_hash': '',
 'timestamp': 1513111888.007075,
 'transactions': []}


In [7]:
print('mining the first block...')
mined_first_block = mine(first_block, miner='tim')
pprint(mined_first_block)

mining the first block...
{'hash': '000079923ac2ee4a292f9aec4b5cb6154174e698609dff282b7f7a3cae0d1e2d',
 'nonce': 155979,
 'previous_hash': '',
 'timestamp': 1513111888.007075,
 'transactions': [{'amount': 1, 'from': 'network', 'to': 'tim'}]}


Next, we create the second block and add our two transactions:

In [8]:
second_block = new_block(mined_first_block)
second_block['transactions'] = [transaction1, transaction2]
print("second block:")
pprint(second_block)

second block:
{'hash': '',
 'nonce': 0,
 'previous_hash': '000079923ac2ee4a292f9aec4b5cb6154174e698609dff282b7f7a3cae0d1e2d',
 'timestamp': 1513111891.667287,
 'transactions': [{'amount': 10, 'from': "A's address", 'to': "B's address"},
                  {'amount': 3, 'from': "B's address", 'to': "C's address"}]}


In [9]:
print('mining the second block...')
mined_second_block = mine(second_block, miner='kevin')
pprint(mined_second_block)

mining the second block...
{'hash': '0000ae6399ef4e762875efefb8c8e3dfb319f2722b80faa1dffa901196b28c60',
 'nonce': 74571,
 'previous_hash': '000079923ac2ee4a292f9aec4b5cb6154174e698609dff282b7f7a3cae0d1e2d',
 'timestamp': 1513111891.667287,
 'transactions': [{'amount': 10, 'from': "A's address", 'to': "B's address"},
                  {'amount': 3, 'from': "B's address", 'to': "C's address"},
                  {'amount': 1, 'from': 'network', 'to': 'kevin'}]}


## The Blockchain

And we can inspect our whole blockchain:

In [10]:
pprint(BLOCKCHAIN)

[{'hash': '000079923ac2ee4a292f9aec4b5cb6154174e698609dff282b7f7a3cae0d1e2d',
  'nonce': 155979,
  'previous_hash': '',
  'timestamp': 1513111888.007075,
  'transactions': [{'amount': 1, 'from': 'network', 'to': 'tim'}]},
 {'hash': '0000ae6399ef4e762875efefb8c8e3dfb319f2722b80faa1dffa901196b28c60',
  'nonce': 74571,
  'previous_hash': '000079923ac2ee4a292f9aec4b5cb6154174e698609dff282b7f7a3cae0d1e2d',
  'timestamp': 1513111891.667287,
  'transactions': [{'amount': 10, 'from': "A's address", 'to': "B's address"},
                   {'amount': 3, 'from': "B's address", 'to': "C's address"},
                   {'amount': 1, 'from': 'network', 'to': 'kevin'}]}]
